# Logistic Regression | Deep Learning

Können wir unser Beispiel auf ein tiefes neuronales Netz übertragen?

<br>
<img src="../img/komplexe-zusammenhaenge.png" width=70%>
<br>

# Ein tiefes neuronales Netz

<br>
<img src="../img/tnn.png" width = "70%">
<br>

In [ ]:
#import some necessary librairies

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)
warnings.warn_explicit = ignore_warn

import pickle
from scipy import stats
from scipy.stats import norm, skew #for some statistics

pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points

from keras.callbacks import ModelCheckpoint, History
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from keras.metrics import RootMeanSquaredError, MeanSquaredError
from matplotlib import pyplot as plt


# Daten aus vorigem Schritt laden

In [ ]:
with open('../data/house-prices-advanced-regression-techniques/x_preprocessed_train.pkl', 'rb') as handle:
    X_preprocessed_train = pickle.load(handle)

with open('../data/house-prices-advanced-regression-techniques/y_train.pkl', 'rb') as handle:
    y_preprocessed_train = pickle.load(handle)
    
with open('../data/house-prices-advanced-regression-techniques/x_test.pkl', 'rb') as handle:
    X_preprocessed_test = pickle.load(handle)


# Train Test Split

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
seed = 4354 # Random_state ist ein seed, damit gegebenenfalls immer mit der selben pseudo Random Folge gearbeitet wird.
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed_train, y_preprocessed_train, test_size=0.2, random_state=seed) 


In [ ]:
print ("Anzahl Datensätze in X_train:\t", X_train.shape[0], "\nVektor der Dimension:\t\t", X_train.shape[1])

In [ ]:
model = Sequential()

# The Input Layer :
model.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

# The Hidden Layers :
model.add(Dense(256, kernel_initializer='normal',activation='relu'))
model.add(Dense(256, kernel_initializer='normal',activation='relu'))
model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
model.compile(loss='mean_squared_error', optimizer='adam', metrics=[MeanSquaredError()])
#model.compile(loss='mean_squared_error', optimizer='adam', metrics=['acc'])
model.summary()

In [ ]:
# Tensorflow allows to creat a visual representation of our model.
from tensorflow.keras.utils import plot_model

plot_model(model, to_file='../img/model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
#checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint_name = 'model.weights.best.hdf5'
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
history = History()
callbacks_list = [checkpoint, history]

In [ ]:
result = model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)


In [ ]:
history.history['mean_squared_error']

In [ ]:
root_mean_squared_error = np.sqrt(history.history['mean_squared_error'])
val_root_mean_squared_error = np.sqrt(history.history['val_mean_squared_error'])

best_epoch = list (val_root_mean_squared_error).index(min(val_root_mean_squared_error)) + 1
print('Best epoch: %d' % (best_epoch,))
print (f'Best RMSE: {min(val_root_mean_squared_error)}')

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(history.history['loss'], color='red', linestyle='--')
ax1.plot(history.history['val_loss'], color='green', linestyle='--')
plt.title('model performance')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train loss', 'val loss', 'train RMSE', 'varl RMSE'], loc='upper left')

ax2 = ax1.twinx()

ax2.plot(root_mean_squared_error)
ax2.plot(val_root_mean_squared_error)
ax2.set_ylabel('RMSE')
ax2.legend(['train RMSE', 'val RMSE'], loc='upper right')

In [ ]:
# Load the weights with the best validation accuracy
model.load_weights('model.weights.best.hdf5')

In [ ]:
# Evaluate the model on test set
score = model.evaluate(X_test, y_test, verbose=0)

# Print test RMSE
print('\n', 'Test RMSE:', np.sqrt(score[1]))

In [ ]:
model.metrics_names 

In [ ]:
y_hat = model.predict(X_test)
y_hat = y_hat.reshape(y_hat.shape[0])

In [ ]:
rand_vals=np.random.choice(X_test.shape[0], size=15, replace=False)
rand_vals

In [ ]:
for i, index in enumerate(rand_vals):
    predict_index = y_hat[index]
    true_index = y_test.to_numpy()[index]
    print (f'{index}: predict={predict_index} / true={true_index}')

In [ ]:
fig, ax1 = plt.subplots()
plt.title('Prediction Delta')
plt.ylabel('Delta')
plt.xlabel('Index')
ax1.plot(np.subtract(y_test.to_numpy(),y_hat), color='red', linestyle='-')